# Transformer

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/datasetTransformed.csv")
df

,Review_Text,Sentiment
0,If you've ever been to Disneyland anywhere you...,0
1,Its been a while since d last time we visit HK...,0
2,Thanks God it wasn t too hot or too humid wh...,0
3,HK Disneyland is a great compact park. Unfortu...,0
4,"the location is not in the city, took around 1...",0
...,...,...
42651,i went to disneyland paris in july 03 and thou...,1
42652,2 adults and 1 child of 11 visited Disneyland ...,1
42653,My eleven year old daughter and myself went to...,1
42654,"This hotel, part of the Disneyland Paris compl...",0


In [ ]:
#IMPORTS
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import classification_report
import tensorflow as tf



# Transformación

In [ ]:
# Cleaning Review_Text column
def text_cleaning(text):
    soup = BeautifulSoup(text, "html.parser")
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())
    pattern = r"[^a-zA-Z0-9\s,']"
    text = re.sub(pattern, '', text)
    return text

df['Clean_text'] = df['Review_Text'].apply(text_cleaning).tolist()

In [ ]:
# Split dataframe into train and test
X = df['Clean_text']
Y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [21]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len= 128
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(X_train.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(X_test.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

# Modelo

In [25]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2,    from_pt=True  
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

NameError: name 'tf' is not defined

# Evaluacion